**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_r_kernel_openblas -p 8889:8888 -d palmoreck/jupyterlab_r_kernel_openblas:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_r_kernel_openblas
```

**Al ejecutar el `run` anterior se descargará la imagen y posteriormente se instalará [OpenBLAS](https://github.com/xianyi/OpenBLAS) y paquetes de Python adaptados a sus sistemas. Esto tardará $10-15$ minutos aproximadamente. Pueden revisar de vez en vez con el commando:**

```
docker logs jupyterlab_r_kernel_openblas
```

**hasta que salga un mensaje del tipo: *...Installed kernelspec ir in /usr/local/share/jupyter/kernels/ir...The Jupyter Notebook is running at...* y poder acceder al puerto 8888 de sus máquinas**


**Nota:** Lo anterior se realizó de esta forma (construir una imagen de docker con software simple y posteriormente instalar el software especializado) pues si hubiera construído la imagen de docker adaptada a mi máquina, es muy probable que se tendría que haber adaptado nuevamente a sus máquinas. Ver: [docker images with architecture-optimisation](https://stackoverflow.com/questions/27919866/docker-images-with-architecture-optimisation) 

Documentación de la imagen de docker `palmoreck/jupyterlab_r_kernel_openblas:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/openblas/r_kernel).

---